# Setup

On some systems, this cell needs to be run twice to get the correct settings of matplotlib.

In [ ]:
from rrk import *
from scipy import interpolate
import time
import scipy as sp

plt.rc("font", family="serif", size=16.)

# Harmonic Oscillator

In [ ]:
def f(w):
    q = w[0]
    p = w[1]
    return np.array([p, -q])

def eta(w):
    q = w[0]
    p = w[1]
    return 0.5*p*p + 0.5*q*q

def d_eta(w):
    fw = f(w)
    dq = fw[0]
    dp = fw[1]
    return np.array([-dp, dq])

def symplectic_euler(dt, w0, t_final):
    w = np.array(w0) # current value of the unknown function
    t = 0 # current time
    ww = np.zeros([np.size(w0), 1]) # values at each time step
    ww[:,0] = w.copy()
    tt = np.zeros(1) # time points for ww
    tt[0] = t

    while t < t_final and not np.isclose(t, t_final):
        if t + dt > t_final:
            dt = t_final - t

        qold = w[0]
        pold = w[1]
        qnew = qold + dt * pold
        pnew = pold + dt * (-qnew)
        w = np.array([qnew, pnew])
        t += dt

        tt = np.append(tt, t)
        ww = np.append(ww, np.reshape(w.copy(), (len(w), 1)), axis=1)

    return tt, ww


t_final = 50.0
t_final_values = np.linspace(0., t_final, 11)[1:]
erk = rk4; dt = 0.25

u0 = np.array([1., 0.0])
num_points = 200 * 2
np.random.seed(42)
radius = np.sqrt(np.random.uniform(0, 0.001**2, num_points))
angle = np.random.uniform(0, 2*np.pi, num_points)
u0_values = np.zeros((num_points, 2))
u0_values[:,0] = radius * np.cos(angle) + u0[0]
u0_values[:,1] = radius * np.sin(angle) + u0[1]
u0_hull = sp.spatial.ConvexHull(u0_values)
vol0 = u0_hull.volume

def phase_space_volume_change(erk, dt, t_final, tts, uus, relaxed):
    u_values = np.zeros_like(u0_values)
    for i in np.arange(num_points):
        # Find the last output time that is less than t_final
        start_ind = np.nonzero(np.where(tts[i]<t_final,1,0))[0].max()
        if relaxed == None:
            tt, uu = symplectic_euler(dt, uus[i][:,start_ind], t_final-tts[i][start_ind])
        else:
#         tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, uus[i][:,start_ind], t_final-tts[i][start_ind],
#                                     relaxed=relaxed, method="brentq", newdt=True)
            tt, uu = relaxed_ERK(erk, dt, f, uus[i][:,start_ind], t_final-tts[i][start_ind],
                                 relaxed=relaxed, newdt=True, gammatol=0.5)
        u_values[i,:] = uu[:,-1]
        
    u_hull = sp.spatial.ConvexHull(u_values)
    rel_change = (u_hull.volume - vol0) / vol0
    
    return rel_change

def precompute_trajectories(erk, dt, t_final):
    # Precompute the trajectories.  Later we'll "interpolate" from these results.
    tts = []
    uus = []
    for i in np.arange(num_points):
#         tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0_values[i,:], t_final,
#                                     relaxed=False, method="brentq", newdt=True)
        tt, uu = relaxed_ERK(erk, dt, f, u0_values[i,:], t_final,
                             relaxed=False, newdt=True, gammatol=0.5)
        tts.append(tt)
        uus.append(uu)

    tts_r = []
    uus_r = []
    for i in np.arange(num_points):
#         tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0_values[i,:], t_final,
#                                     relaxed=True, method="brentq", newdt=True)
        tt, uu = relaxed_ERK(erk, dt, f, u0_values[i,:], t_final,
                             relaxed=True, newdt=True, gammatol=0.5)
        tts_r.append(tt)
        uus_r.append(uu)
        
    tts_s = []
    uus_s = []
    for i in np.arange(num_points):
        tt, uu = symplectic_euler(dt, u0_values[i,:], t_final)
        tts_s.append(tt)
        uus_s.append(uu)
    
    return tts, uus, tts_r, uus_r, tts_s, uus_s
    
tts, uus, tts_r, uus_r, tts_s, uus_s = precompute_trajectories(erk, dt, t_final)
baseline_rel_change = np.zeros_like(t_final_values)
relaxation_rel_change = np.zeros_like(t_final_values)
symplectic_rel_change = np.zeros_like(t_final_values)

for (t_final_idx, t_final) in enumerate(t_final_values):
    baseline_rel_change[t_final_idx] = phase_space_volume_change(erk, dt, t_final, tts, uus, relaxed=False)
    relaxation_rel_change[t_final_idx] = phase_space_volume_change(erk, dt, t_final, tts_r, uus_r, relaxed=True)
    symplectic_rel_change[t_final_idx] = phase_space_volume_change(erk, dt, t_final, tts_s, uus_s, relaxed=None)

plt.close("all")
fig, ax = plt.subplots(1, 1)
ax.set_prop_cycle(marker_cycler)
plt.plot(t_final_values, baseline_rel_change, label="Baseline")
plt.plot(t_final_values, relaxation_rel_change, label="Relaxation")
plt.plot(t_final_values, symplectic_rel_change, label="Symplectic Euler")
plt.xlabel(r"$t$")
plt.ylabel("Rel. Change of Phase Space Vol.")
plt.legend(loc="best")
plt.savefig("../figures/phase_space_volume__RK4__linear_osc.pdf", bbox_inches="tight")


print(np.linalg.lstsq(t_final_values.reshape(-1,1), baseline_rel_change, rcond=None))
print(np.linalg.lstsq(t_final_values.reshape(-1,1), relaxation_rel_change, rcond=None))
print(np.linalg.lstsq(t_final_values.reshape(-1,1), symplectic_rel_change, rcond=None))
print(vol0)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_aspect("equal"); xlim = plt.xlim(); ylim = plt.ylim()
time_tmp = time.time()
for idx in np.arange(len(uus)):
    plt.scatter(uus[idx][0,-1], uus[idx][1,-1], color="blue", marker=".")

fig, ax = plt.subplots(1, 1)
ax.set_aspect("equal"); xlim = plt.xlim(); ylim = plt.ylim()
time_tmp = time.time()
for idx in np.arange(len(uus_r)):
    plt.scatter(uus_r[idx][0,-1], uus_r[idx][1,-1], color="blue", marker=".")

fig, ax = plt.subplots(1, 1)
ax.set_aspect("equal"); xlim = plt.xlim(); ylim = plt.ylim()
time_tmp = time.time()
for idx in np.arange(len(uus_s)):
    plt.scatter(uus_s[idx][0,-1], uus_s[idx][1,-1], color="blue", marker=".")

# Two Harmonic Oscillators

In [ ]:
def f(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    return np.array([p1, p2, -q1, -q2])

def eta(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    return 0.5 * (p1*p1 + p2*p2 + q1*q1 + q2*q2)

def d_eta(w):
    fw = f(w)
    dq1 = fw[0]
    dq2 = fw[1]
    dp1 = fw[2]
    dp2 = fw[3]
    return np.array([-dp1, -dp2, dq1, dq2])

def symplectic_euler(dt, w0, t_final):
    w = np.array(w0) # current value of the unknown function
    t = 0 # current time
    ww = np.zeros([np.size(w0), 1]) # values at each time step
    ww[:,0] = w.copy()
    tt = np.zeros(1) # time points for ww
    tt[0] = t

    while t < t_final and not np.isclose(t, t_final):
        if t + dt > t_final:
            dt = t_final - t

        q1old = w[0]
        q2old = w[1]
        p1old = w[2]
        p2old = w[3]
        q1new = q1old + dt * p1old
        q2new = q2old + dt * p2old
        p1new = p1old + dt * (-q1new)
        p2new = p2old + dt * (-q2new)
        w = np.array([q1new, q2new, p1new, p2new])
        t += dt

        tt = np.append(tt, t)
        ww = np.append(ww, np.reshape(w.copy(), (len(w), 1)), axis=1)

    return tt, ww


t_final = 50.0
t_final_values = np.linspace(0., t_final, 11)[1:]
erk = rk4; dt = 0.25

u0 = np.array([1.0, 0.0, 0.5, 0.5])
num_points = 100 * 1
np.random.seed(42)
u0_values = np.zeros((num_points, 4))
u0_values[:,0] = u0[0] + np.random.uniform(-0.0005, 0.0005, num_points)
u0_values[:,1] = u0[1] + np.random.uniform(-0.0005, 0.0005, num_points)
u0_values[:,2] = u0[2] + np.random.uniform(-0.0005, 0.0005, num_points)
u0_values[:,3] = u0[3] + np.random.uniform(-0.0005, 0.0005, num_points)
u0_hull = sp.spatial.ConvexHull(u0_values)
vol0 = u0_hull.volume

def phase_space_volume_change(erk, dt, t_final, tts, uus, relaxed):
    u_values = np.zeros_like(u0_values)
    for i in np.arange(num_points):
        # Find the last output time that is less than t_final
        start_ind = np.nonzero(np.where(tts[i]<t_final,1,0))[0].max()
        if relaxed == None:
            tt, uu = symplectic_euler(dt, uus[i][:,start_ind], t_final-tts[i][start_ind])
        else:
#         tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, uus[i][:,start_ind], t_final-tts[i][start_ind],
#                                     relaxed=relaxed, method="brentq", newdt=True)
            tt, uu = relaxed_ERK(erk, dt, f, uus[i][:,start_ind], t_final-tts[i][start_ind],
                                 relaxed=relaxed, newdt=True, gammatol=0.5)
        u_values[i,:] = uu[:,-1]
        
    u_hull = sp.spatial.ConvexHull(u_values)
    rel_change = (u_hull.volume - vol0) / vol0
    
    return rel_change

def precompute_trajectories(erk, dt, t_final):
    # Precompute the trajectories.  Later we'll "interpolate" from these results.
    tts = []
    uus = []
    for i in np.arange(num_points):
#         tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0_values[i,:], t_final,
#                                     relaxed=False, method="brentq", newdt=True)
        tt, uu = relaxed_ERK(erk, dt, f, u0_values[i,:], t_final,
                             relaxed=False, newdt=True, gammatol=0.5)
        tts.append(tt)
        uus.append(uu)

    tts_r = []
    uus_r = []
    for i in np.arange(num_points):
#         tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0_values[i,:], t_final,
#                                     relaxed=True, method="brentq", newdt=True)
        tt, uu = relaxed_ERK(erk, dt, f, u0_values[i,:], t_final,
                             relaxed=True, newdt=True, gammatol=0.5)
        tts_r.append(tt)
        uus_r.append(uu)
        
    tts_s = []
    uus_s = []
    for i in np.arange(num_points):
        tt, uu = symplectic_euler(dt, u0_values[i,:], t_final)
        tts_s.append(tt)
        uus_s.append(uu)
    
    return tts, uus, tts_r, uus_r, tts_s, uus_s
    
tts, uus, tts_r, uus_r, tts_s, uus_s = precompute_trajectories(erk, dt, t_final)
baseline_rel_change = np.zeros_like(t_final_values)
relaxation_rel_change = np.zeros_like(t_final_values)
symplectic_rel_change = np.zeros_like(t_final_values)

for (t_final_idx, t_final) in enumerate(t_final_values):
    baseline_rel_change[t_final_idx] = phase_space_volume_change(erk, dt, t_final, tts, uus, relaxed=False)
    relaxation_rel_change[t_final_idx] = phase_space_volume_change(erk, dt, t_final, tts_r, uus_r, relaxed=True)
    symplectic_rel_change[t_final_idx] = phase_space_volume_change(erk, dt, t_final, tts_s, uus_s, relaxed=None)

plt.close("all")
fig, ax = plt.subplots(1, 1)
ax.set_prop_cycle(marker_cycler)
plt.plot(t_final_values, baseline_rel_change, label="Baseline")
plt.plot(t_final_values, relaxation_rel_change, label="Relaxation")
plt.plot(t_final_values, symplectic_rel_change, label="Symplectic Euler")
plt.xlabel(r"$t$")
plt.ylabel("Rel. Change of Phase Space Vol.")
plt.legend(loc="best")
plt.savefig("../figures/phase_space_volume__RK4__linear_osc_2.pdf", bbox_inches="tight")


print(np.linalg.lstsq(t_final_values.reshape(-1,1), baseline_rel_change, rcond=None))
print(np.linalg.lstsq(t_final_values.reshape(-1,1), relaxation_rel_change, rcond=None))
print(np.linalg.lstsq(t_final_values.reshape(-1,1), symplectic_rel_change, rcond=None))
print(vol0)

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].set_aspect("equal")
for idx in np.arange(len(uus)):
    ax[0].scatter(uus[idx][0,-1], uus[idx][2,-1], color="black", marker=".")
ax[0].set_xlabel(r"$q_1$")
ax[0].set_ylabel(r"$p_1$")
ax[1].set_aspect("equal")
for idx in np.arange(len(uus)):
    ax[1].scatter(uus[idx][1,-1], uus[idx][3,-1], color="black", marker=".")
ax[1].set_xlabel(r"$q_2$")
ax[1].set_ylabel(r"$p_2$")
plt.tight_layout()
plt.suptitle("Baseline")

fig, ax = plt.subplots(1, 2)
ax[0].set_aspect("equal")
for idx in np.arange(len(uus_r)):
    ax[0].scatter(uus_r[idx][0,-1], uus_r[idx][2,-1], color="black", marker=".")
ax[0].set_xlabel(r"$q_1$")
ax[0].set_ylabel(r"$p_1$")
ax[1].set_aspect("equal")
for idx in np.arange(len(uus_r)):
    ax[1].scatter(uus_r[idx][1,-1], uus_r[idx][3,-1], color="black", marker=".")
ax[1].set_xlabel(r"$q_2$")
ax[1].set_ylabel(r"$p_2$")
plt.tight_layout()
plt.suptitle("Relaxation")

fig, ax = plt.subplots(1, 2)
ax[0].set_aspect("equal")
for idx in np.arange(len(uus_s)):
    ax[0].scatter(uus_s[idx][0,-1], uus_s[idx][2,-1], color="black", marker=".")
ax[0].set_xlabel(r"$q_1$")
ax[0].set_ylabel(r"$p_1$")
ax[0].set_xlim(
    np.min([uus_s[idx][0,-1] for idx in np.arange(len(uus_s))]), 
    np.max([uus_s[idx][0,-1] for idx in np.arange(len(uus_s))]))
ax[0].set_ylim(
    np.min([uus_s[idx][2,-1] for idx in np.arange(len(uus_s))]), 
    np.max([uus_s[idx][2,-1] for idx in np.arange(len(uus_s))]))
ax[1].set_aspect("equal")
for idx in np.arange(len(uus_s)):
    ax[1].scatter(uus_s[idx][1,-1], uus_s[idx][3,-1], color="black", marker=".")
ax[1].set_xlabel(r"$q_2$")
ax[1].set_ylabel(r"$p_2$")
ax[1].set_xlim(
    np.min([uus_s[idx][1,-1] for idx in np.arange(len(uus_s))]), 
    np.max([uus_s[idx][1,-1] for idx in np.arange(len(uus_s))]))
ax[1].set_ylim(
    np.min([uus_s[idx][3,-1] for idx in np.arange(len(uus_s))]), 
    np.max([uus_s[idx][3,-1] for idx in np.arange(len(uus_s))]))
plt.tight_layout()
plt.suptitle("Symplectic Euler")

# Undamped Duffing Oscillator

In [ ]:
def f(w):
    q = w[0]
    p = w[1]
    return np.array([p, q - q*q*q])

def eta(w):
    q = w[0]
    p = w[1]
    return 0.5*p*p - 0.5*q*q + 0.25*q*q*q*q

def d_eta(w):
    fw = f(w)
    dq = fw[0]
    dp = fw[1]
    return np.array([-dp, dq])

def symplectic_euler(dt, w0, t_final):
    w = np.array(w0) # current value of the unknown function
    t = 0 # current time
    ww = np.zeros([np.size(w0), 1]) # values at each time step
    ww[:,0] = w.copy()
    tt = np.zeros(1) # time points for ww
    tt[0] = t

    while t < t_final and not np.isclose(t, t_final):
        if t + dt > t_final:
            dt = t_final - t

        qold = w[0]
        pold = w[1]
        qnew = qold + dt * pold
        pnew = pold + dt * (qnew - qnew*qnew*qnew)
        w = np.array([qnew, pnew])
        t += dt

        tt = np.append(tt, t)
        ww = np.append(ww, np.reshape(w.copy(), (len(w), 1)), axis=1)

    return tt, ww


t_final = 200
t_final_values = np.linspace(0., t_final, 11)[1:]
# erk = rk4; dt = 0.25
# erk = rk4; dt = 0.4
erk = ssp33; dt = 0.1

u0 = np.array([1., 0.0])
num_points = 200 * 1
np.random.seed(42)
radius = np.sqrt(np.random.uniform(0, 0.001**2, num_points))
angle = np.random.uniform(0, 2*np.pi, num_points)
u0_values = np.zeros((num_points, 2))
u0_values[:,0] = radius * np.cos(angle) + u0[0]
u0_values[:,1] = radius * np.sin(angle) + u0[1]
u0_hull = sp.spatial.ConvexHull(u0_values)
vol0 = u0_hull.volume

def phase_space_volume_change(erk, dt, t_final, tts, uus, relaxed):
    u_values = np.zeros_like(u0_values)
    for i in np.arange(num_points):
        # Find the last output time that is less than t_final
        start_ind = np.nonzero(np.where(tts[i]<t_final,1,0))[0].max()
        if relaxed == None:
            tt, uu = symplectic_euler(dt, uus[i][:,start_ind], t_final-tts[i][start_ind])
        else:
            tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, uus[i][:,start_ind], t_final-tts[i][start_ind],
                                        relaxed=relaxed, method="brentq", newdt=True)
#              tt, uu = relaxed_ERK(erk, dt, f, uus[i][:,start_ind], t_final-tts[i][start_ind],
#                                  relaxed=relaxed, newdt=True, gammatol=0.5)
        u_values[i,:] = uu[:,-1]
        
    u_hull = sp.spatial.ConvexHull(u_values)
    rel_change = (u_hull.volume - vol0) / vol0
    
    return rel_change

def precompute_trajectories(erk, dt, t_final):
    # Precompute the trajectories.  Later we'll "interpolate" from these results.
    tts = []
    uus = []
    for i in np.arange(num_points):
        tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0_values[i,:], t_final,
                                    relaxed=False, method="brentq", newdt=True)
#         tt, uu = relaxed_ERK(erk, dt, f, u0_values[i,:], t_final,
#                              relaxed=False, newdt=True, gammatol=0.5)
        tts.append(tt)
        uus.append(uu)

    tts_r = []
    uus_r = []
    for i in np.arange(num_points):
        tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0_values[i,:], t_final,
                                    relaxed=True, method="brentq", newdt=True)
#         tt, uu = relaxed_ERK(erk, dt, f, u0_values[i,:], t_final,
#                              relaxed=True, newdt=True, gammatol=0.5)
        tts_r.append(tt)
        uus_r.append(uu)
        
    tts_s = []
    uus_s = []
    for i in np.arange(num_points):
        tt, uu = symplectic_euler(dt, u0_values[i,:], t_final)
        tts_s.append(tt)
        uus_s.append(uu)
    
    return tts, uus, tts_r, uus_r, tts_s, uus_s
    
tts, uus, tts_r, uus_r, tts_s, uus_s = precompute_trajectories(erk, dt, t_final)
baseline_rel_change = np.zeros_like(t_final_values)
relaxation_rel_change = np.zeros_like(t_final_values)
symplectic_rel_change = np.zeros_like(t_final_values)

for (t_final_idx, t_final) in enumerate(t_final_values):
    baseline_rel_change[t_final_idx] = phase_space_volume_change(erk, dt, t_final, tts, uus, relaxed=False)
    relaxation_rel_change[t_final_idx] = phase_space_volume_change(erk, dt, t_final, tts_r, uus_r, relaxed=True)
    symplectic_rel_change[t_final_idx] = phase_space_volume_change(erk, dt, t_final, tts_s, uus_s, relaxed=None)

plt.close("all")
fig, ax = plt.subplots(1, 1)
ax.set_prop_cycle(marker_cycler)
plt.plot(t_final_values, baseline_rel_change, label="Baseline")
plt.plot(t_final_values, relaxation_rel_change, label="Relaxation")
plt.plot(t_final_values, symplectic_rel_change, label="Symplectic Euler")
plt.xlabel(r"$t$")
plt.ylabel("Rel. Change of Phase Space Vol.")
plt.legend(loc="best")
plt.savefig("../figures/phase_space_volume__SSP33__duffing_osc.pdf", bbox_inches="tight")


print(np.linalg.lstsq(t_final_values.reshape(-1,1), baseline_rel_change, rcond=None))
print(np.linalg.lstsq(t_final_values.reshape(-1,1), relaxation_rel_change, rcond=None))
print(np.linalg.lstsq(t_final_values.reshape(-1,1), symplectic_rel_change, rcond=None))
print(vol0)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_aspect("equal"); xlim = plt.xlim(); ylim = plt.ylim()
time_tmp = time.time()
for idx in np.arange(len(uus)):
    plt.scatter(uus[idx][0,-1], uus[idx][1,-1], color="blue", marker=".")

fig, ax = plt.subplots(1, 1)
ax.set_aspect("equal"); xlim = plt.xlim(); ylim = plt.ylim()
time_tmp = time.time()
for idx in np.arange(len(uus_r)):
    plt.scatter(uus_r[idx][0,-1], uus_r[idx][1,-1], color="blue", marker=".")